## Yahoo Fundamental

In [15]:
# def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
#     """
#     Extrat data from trading economics callendar.

#     Parameters:
#         classe_tabela (str): Class CSS from table to scrap (optional).

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')

#         if table_class:
#             tabela = soup.find('table', {'class': table_class})
#         else:
#             tabela = soup.find('table')
        
#         headers = [th.text.strip() for th in tabela.find_all('th')]
#         print(headers)
#         rows = []
#         for row in tabela.find_all('tr')[1:]:
#             cols = [td.text.strip() for td in row.find_all('td')]
#             if cols: 
#                 rows.append(cols)

#         df = pd.DataFrame(rows, columns=headers if headers else None)
        

#     except requests.exceptions.RequestException as e:
#         print(f"Error accessing URL: {e}")
#     except Exception as e:
#         print(f"Error processing data: {e}")

#     return df


In [1]:
# def get_whalewisdom(ticker: str) -> pd.DataFrame:
#     """
#     Extracts data from WhaleWisdom for a given stock ticker.

#     Parameters:
#         ticker (str): Stock ticker symbol.

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Encontrar a div com a classe "v-data-table__wrapper"
#         div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
#         if not div_wrapper:
#             raise ValueError("Div com a classe 'v-window__container' não encontrada.")

#         # Buscar a primeira tabela dentro desta div
#         tabela = div_wrapper.find('table')
#         if not tabela:
#             raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
#         # Coletar os cabeçalhos da tabela ignorando ícones
#         headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
#         # print(headers)

#         # Encontrando todas as linhas dentro da tabela (tbody > tr)
#         rows = tabela.find("tbody").find_all("tr")
#         print(rows)
#         # Extraindo os dados de cada linha
#         data = []
#         for row in rows:
#             columns = row.find_all("td")

#             # Pegando os valores correspondentes
#             institution_name = columns[0].text.strip()
#             shares_held = columns[2].text.strip()
#             portfolio_value = columns[3].text.strip()
#             percentage_ownership = columns[4].text.strip()
#             last_report_date = columns[-1].text.strip()

#             data.append({
#                 "Instituição": institution_name,
#                 "Ações Detidas": shares_held,
#                 "Valor do Portfólio": portfolio_value,
#                 "Percentual de Participação": percentage_ownership,
#                 "Última Atualização": last_report_date
#             })
#         #print(data)
    
#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao acessar a URL: {e}")
#     except ValueError as e:
#         print(f"Erro na extração de dados: {e}")
#     except Exception as e:
#         print(f"Erro inesperado: {e}")
    
#     return pd.DataFrame()

In [2]:
# df_whale_s = get_whalewisdom(ticker = "ctre")
# df_whale_s

In [3]:
# base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Accept": "application/json"
# }

# response = requests.get(base_url, headers=headers)
# response.raise_for_status()

# response.text

In [4]:
# import hashlib
# import hmac
# import base64
# import json
# import time
# import urllib.parse
# import requests

# class WhaleWisdom:
#     def __init__(self, secret_key, shared_key):
#         self.secret_key = secret_key
#         self.shared_key = shared_key
#         self.timestamp = self.get_timestamp()

#     def get_timestamp(self):
#         """Gera o timestamp no formato UTC"""
#         return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

#     def signature(self, args):
#         """Gera a assinatura HMAC-SHA1 em Base64"""
#         message = f"{args}\n{self.timestamp}".encode('utf-8')
#         hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
#         return base64.b64encode(hmac_digest).decode('utf-8')

#     def encode(self, string):
#         """Codifica a string para URL"""
#         return urllib.parse.quote(string, safe='')

#     def endpoint(self, args):
#         """Constrói a URL da API do WhaleWisdom"""
#         encoded_args = self.encode(json.dumps(args))
#         api_sig = self.signature(json.dumps(args))
#         encoded_timestamp = self.encode(self.timestamp)

#         return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

#     def request(self, args):
#         """Faz a requisição à API"""
#         url = self.endpoint(args)
#         response = requests.get(url)

#         if response.status_code == 200:
#             return response.text
#         else:
#             print(f"Erro {response.status_code}: {response.text}")
#             return None


# # **Exemplo de Uso**
# WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
# WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

# ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# # Obter as participações (holdings) de um gestor específico
# response = ww.request({
#     "command": "stock_lookup",
#     "symbol": "ctre"
# })

# data = json.loads(response)

# stock_id = data["stocks"][0]["id"]
# name = data["stocks"][0]["name"]
# status = data["stocks"][0]["status"]
# link = data["stocks"][0]["link"]

# stock_id

In [5]:
# response = ww.request(
# {"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
# response


In [6]:
# response

## STOCKS



In [59]:
import yfinance as yf
import pandas as pd
import talib
import numpy as np
from django.http import JsonResponse
from datetime import datetime, timedelta

# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "TNXP"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários

yahoo_symbol_info = yf.Ticker(symbol).info

In [61]:
yahoo_symbol_info.get("trailingPE", "N/A")

'N/A'

In [9]:
def get_sector_etf_info(sector_name, search_value: str = "info" ):
    """
    Return information about symbol ETF sector.
    """
    sector_map = {
        "Technology": "XLK",
        "Financial Services": "XLF",
        "Consumer Cyclical": "XLY",
        "Healthcare": "XLV",
        "Communication Services": "XLC",
        "Industrials": "XLI",
        "Consumer Defensive": "XLP",
        "Energy": "XLE",
        "Real Estate": "XLRE",
        "Basic Materials": "XLB",
        "Utilities": "XLU",
    }

    etf_symbol = sector_map.get(sector_name)
    if not etf_symbol:
        return "N/A"

    ticker = yf.Ticker(etf_symbol)

    if search_value == "info":
        return ticker.info
    else:
        return ticker.info.get(search_value, "N/A")


In [49]:
def get_yoy_metric(series):
    if isinstance(series, pd.Series):
        series = series.replace(0, np.nan).dropna()

        if len(series) >= 2:
            yoy_growth = ((series - series.shift(-1)) / series.shift(-1)) * 100
            yoy_growth = yoy_growth.dropna()
        else:
            yoy_growth = pd.Series("N/A", index=series.index)

    return yoy_growth


def get_cagr_metric(series):
    if isinstance(series, pd.Series):
        series = pd.to_numeric(series, errors='coerce').dropna()

        if len(series) >= 2:
            start_value = series.iloc[-1]
            end_value = series.iloc[0]
            num_years = len(series) - 1

            if start_value != 0:
                cagr = ((end_value / start_value) ** (1 / num_years)) - 1
            else:
                cagr = "N/A"
        else:
            cagr = "N/A"
    else:
        cagr = "N/A"

    cagr_percent = cagr * 100 if cagr != "N/A" else "N/A"

    return cagr_percent

In [50]:
yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
yahoo_symbol_cashflow.index

Index(['Free Cash Flow', 'Repayment Of Debt', 'Issuance Of Debt',
       'Issuance Of Capital Stock', 'Capital Expenditure', 'End Cash Position',
       'Other Cash Adjustment Outside Changein Cash',
       'Beginning Cash Position', 'Effect Of Exchange Rate Changes',
       'Changes In Cash', 'Financing Cash Flow',
       'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Proceeds From Stock Option Exercised',
       'Cash Dividends Paid', 'Common Stock Dividend Paid',
       'Net Preferred Stock Issuance', 'Preferred Stock Issuance',
       'Net Common Stock Issuance', 'Common Stock Issuance',
       'Net Issuance Payments Of Debt', 'Net Long Term Debt Issuance',
       'Long Term Debt Payments', 'Long Term Debt Issuance',
       'Investing Cash Flow', 'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Net Investment Purchase And Sale',
       'Sale Of Investment', 'Purchase Of Investment',
       'Net Busine

In [51]:
yahoo_symbol_income = yf.Ticker(symbol).income_stmt
yahoo_symbol_income.index

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Preferred Stock Dividends',
       'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
       'Special Income Charges', 'Gain On Sa

In [52]:
yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
yahoo_symbol_balancesheet.index

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Preferred Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Minority Interest',
       'Stockholders Equity', 'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Foreign Currency Translation Adjustments',
       'Minimum Pension Liabilities', 'Treasury Stock', 'Retained Earnings',
       'Additional Paid In Capital', 'Capital Stock', 'Common Stock',
       'Preferred Stock', 'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Employee Benefits',
       'Non Current Pension And Other Postretirement Benefit Plans',
       'Non Current Deferred Liabilities',
       'Non Cur

In [53]:
# current_assets
if 'Current Assets' in yahoo_symbol_balancesheet.index:
    current_assets = yahoo_symbol_balancesheet.loc['Current Assets']
    if pd.isna(current_assets).all():
        current_assets = "N/A"

if 'Current Liabilities' in yahoo_symbol_balancesheet.index:
    current_liabilities = yahoo_symbol_balancesheet.loc['Current Liabilities']
    if pd.isna(current_liabilities).all():
        current_liabilities = "N/A"

In [54]:
current_assets,current_liabilities, (current_assets / current_liabilities).where(current_liabilities != 0) * 100

(2024-12-31    127998000000.0
 2023-12-31    109275000000.0
 2022-12-31    109523000000.0
 2021-12-31    108666000000.0
 2020-12-31               NaN
 Name: Current Assets, dtype: object,
 2024-12-31    97078000000.0
 2023-12-31    95827000000.0
 2022-12-31    90052000000.0
 2021-12-31    81992000000.0
 2020-12-31              NaN
 Name: Current Liabilities, dtype: object,
 2024-12-31    131.850677
 2023-12-31    114.033623
 2022-12-31    121.621952
 2021-12-31    132.532442
 2020-12-31           NaN
 dtype: object)

In [57]:
def get_symbol_fundamental_info(symbol : str):
        '''
        Return detailed fundamental information about asset
        '''
        try:
            yahoo_symbol_info = yf.Ticker(symbol).info
        except:
            yahoo_symbol_info = {}
        try:
            yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
        except:
            yahoo_symbol_balancesheet = pd.DataFrame()
        try:
            yahoo_symbol_income = yf.Ticker(symbol).income_stmt
        except:
            yahoo_symbol_income = pd.DataFrame()
        try:
            yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
        except:
            yahoo_symbol_cashflow = pd.DataFrame()
        
        # Valuation
        sector = yahoo_symbol_info.get("sector")

        sector_pe = get_sector_etf_info(sector, "trailingPE")

        # Dividends & BuyBacks
        eps_ann = yahoo_symbol_info.get("epsCurrentYear", "N/A")
        dividend_rate = yahoo_symbol_info.get("dividendRate", "N/A")
        if eps_ann != "N/A" and dividend_rate != "N/A" and dividend_rate != 0:
            div_coverage_rate = eps_ann / dividend_rate
        else:
            div_coverage_rate = "N/A" 

        # Profitability
        # net_income
        if 'Net Income' in yahoo_symbol_income.index:
            net_income = yahoo_symbol_income.loc['Net Income']
            if pd.isna(net_income).all():
                net_income = "N/A"

        # total_revenue
        if 'Total Revenue' in yahoo_symbol_income.index:
            total_revenue = yahoo_symbol_income.loc['Total Revenue']
            if pd.isna(total_revenue).all():
                total_revenue = "N/A"

        # cost_of_revenue
        if 'Cost Of Revenue' in yahoo_symbol_income.index:
            cost_of_revenue = yahoo_symbol_income.loc['Cost Of Revenue']
            if pd.isna(cost_of_revenue).all():
                cost_of_revenue = "N/A"

        # gross_profit
        if 'Gross Profit' in yahoo_symbol_income.index:
            gross_profit = yahoo_symbol_income.loc['Gross Profit']
            if pd.isna(gross_profit).all():
                gross_profit = "N/A"

        # operating_expenses
        if 'Operating Expense' in yahoo_symbol_income.index:
            operating_expenses = yahoo_symbol_income.loc['Operating Expense']
            if pd.isna(operating_expenses).all():
                operating_expenses = "N/A"
                            
        yoy_cost_of_revenue = get_yoy_metric(cost_of_revenue)
        yoy_total_revenue = get_yoy_metric(total_revenue)
        yoy_operating_expenses = get_yoy_metric(operating_expenses)

        cagr_cost_of_revenue = get_cagr_metric(cost_of_revenue)
        cagr_total_revenue = get_cagr_metric(total_revenue)
        cagr_operating_expenses = get_cagr_metric(operating_expenses)

        # Growth & NetWorth & Health
        # total_assets
        if 'Total Assets' in yahoo_symbol_balancesheet.index:
            total_assets = yahoo_symbol_balancesheet.loc['Total Assets']
            if pd.isna(total_assets).all():
                total_assets = "N/A"
        
        # current_liabilities
        if 'Current Liabilities' in yahoo_symbol_balancesheet.index:
            current_liabilities = yahoo_symbol_balancesheet.loc['Current Liabilities']
            if pd.isna(current_liabilities).all():
                current_liabilities = "N/A"

        # non_current_liabilities
        if 'Total Non Current Liabilities Net Minority Interest' in yahoo_symbol_balancesheet.index:
            non_current_liabilities = yahoo_symbol_balancesheet.loc['Total Non Current Liabilities Net Minority Interest']
            if pd.isna(non_current_liabilities).all():
                non_current_liabilities = "N/A"

        # total_liabilities
        if not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
            total_liabilities = (current_liabilities + non_current_liabilities)
        else:
            total_liabilities = "N/A"

        # net_worth
        if not total_assets.isna().all() and not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
            total_assets = total_assets.replace(0, np.nan)
            net_worth = total_assets.iloc[0] - total_liabilities.iloc[0]
        else:
            net_worth = "N/A"

        # current_assets
        if 'Current Assets' in yahoo_symbol_balancesheet.index:
            current_assets = yahoo_symbol_balancesheet.loc['Current Assets']
            if pd.isna(current_assets).all():
                current_assets = "N/A"

        # non_current_assets
        if 'Total Non Current Assets' in yahoo_symbol_balancesheet.index:
            non_current_assets = yahoo_symbol_balancesheet.loc['Total Non Current Assets']
            if pd.isna(non_current_assets).all():
                non_current_assets = "N/A"
        
        # short_term_debt_coverage
        if not current_assets.isna().all() and not current_liabilities.isna().all():
            short_term_debt_coverage = current_assets - current_liabilities
        else:
            short_term_debt_coverage = "N/A"

        # long_term_debt_coverage
        if not non_current_assets.isna().all() and not non_current_liabilities.isna().all():
            long_term_debt_coverage = non_current_assets - non_current_liabilities
        else:
            long_term_debt_coverage = "N/A"

        cagr_total_liabilities = get_cagr_metric(total_liabilities)
        cagr_total_assets = get_cagr_metric(total_assets)

        # cash_cash_equivalents
        if 'Cash And Cash Equivalents' in yahoo_symbol_balancesheet.index:
            cash_cash_equivalents = yahoo_symbol_balancesheet.loc['Cash And Cash Equivalents']
            if pd.isna(cash_cash_equivalents).all():
                cash_cash_equivalents = "N/A"

        # stockholders_equity
        if 'Stockholders Equity' in yahoo_symbol_balancesheet.index:
            stockholders_equity = yahoo_symbol_balancesheet.loc['Stockholders Equity']
            if pd.isna(stockholders_equity).all():
                stockholders_equity = "N/A"
        
        cagr_stockholder_equity = get_cagr_metric(stockholders_equity)
        
        # Cashflow
        # free_cashflow
        if 'Free Cash Flow' in yahoo_symbol_cashflow.index:
            free_cashflow = yahoo_symbol_cashflow.loc['Free Cash Flow']
            if pd.isna(free_cashflow).all():
                free_cashflow = "N/A"
        
        # operating_cashflow
        if 'Operating Cash Flow' in yahoo_symbol_cashflow.index:
            operating_cashflow = yahoo_symbol_cashflow.loc['Operating Cash Flow']
            if pd.isna(operating_cashflow).all():
                operating_cashflow = "N/A"

        # capital_expenditure
        if 'Capital Expenditure' in yahoo_symbol_cashflow.index:
            capital_expenditure = yahoo_symbol_cashflow.loc['Capital Expenditure']
            if pd.isna(capital_expenditure).all():
                capital_expenditure = "N/A"

        # market_cap
        market_cap = yahoo_symbol_info.get('marketCap', "N/A")

        # free_cashflow_yield
        if not pd.isna(market_cap) and market_cap != 0 and not free_cashflow.isna().all():
            free_cashflow_yield = ((free_cashflow.iloc[0] / market_cap) * 100)
        else:
            free_cashflow_yield = "N/A"

        # Ratios
        # current_ratio
        if not current_assets.isna().all() and not current_liabilities.isna().all():
            current_liabilities = current_liabilities.replace(0, np.nan)
            current_ratio = ((current_assets / current_liabilities) * 100).fillna("N/A")
        else:
            current_ratio = "N/A"
        
        # cagr_current_ratio
        cagr_current_ratio = get_cagr_metric(current_ratio)
        
        # cash_ratio
        if not cash_cash_equivalents.isna().all() and not current_liabilities.isna().all():
            current_liabilities = current_liabilities.replace(0, np.nan)
            cash_ratio = ((cash_cash_equivalents / current_liabilities) * 100).fillna("N/A")
        else:
            cash_ratio = "N/A"
        
        # cagr_cash_ratio
        cagr_cash_ratio = get_cagr_metric(cash_ratio)

        # gross_margin
        if not gross_profit.isna().all() and not total_revenue.isna().all():
            total_revenue = total_revenue.replace(0, np.nan)
            gross_margin = ((gross_profit / total_revenue) * 100).fillna("N/A")
        else:
            gross_margin = "N/A"

        # cagr_gross_margin
        # cagr_gross_margin = fm.get_cagr_metric(gross_margin)

        # operating_income
        if 'Operating Income' in yahoo_symbol_income.index:
            operating_income = yahoo_symbol_income.loc['Operating Income']
            if pd.isna(operating_income).all():
                operating_income = "N/A"

        # operation_margin
        if not operating_income.isna().all() and not total_revenue.isna().all():
            total_revenue = total_revenue.replace(0, np.nan)
            operating_margin = ((operating_income / total_revenue) * 100).fillna("N/A")
        else:
            operating_margin = "N/A"

        # cagr_operating_margin
        cagr_operating_margin = get_cagr_metric(operating_margin)

        # profit_margin
        if not net_income.isna().all() and not total_revenue.isna().all():
            total_revenue = total_revenue.replace(0, np.nan)
            profit_margin = ((net_income / total_revenue) * 100).fillna("N/A")
        else:
            profit_margin = "N/A"

        # cagr_profit_margin
        cagr_profit_margin = get_cagr_metric(profit_margin)
        
        # return_on_equity
        if not net_income.isna().all() and not stockholders_equity.isna().all():
            stockholders_equity = stockholders_equity.replace(0, np.nan)
            return_on_equity = ((net_income / stockholders_equity) * 100).fillna("N/A")
        else:
            return_on_equity = "N/A"

        # cagr_return_on_equity
        cagr_return_on_equity = get_cagr_metric(return_on_equity)
        
        yahoo_symbol_fundamental_info = {
            "valuation": {
                "trailingPE": yahoo_symbol_info.get("trailingPE", "N/A"),
                "sectorTrailingPE": sector_pe,
                "forwardPE": yahoo_symbol_info.get("forwardPE", "N/A"),
                "PEGRatio": yahoo_symbol_info.get("trailingPegRatio", "N/A"),
            },
            "dividends": {
                "divCoverageRate": div_coverage_rate,
                "dividendYield": yahoo_symbol_info.get("dividendYield", "N/A"),
                "fiveYearAvgDividendYield": yahoo_symbol_info.get("fiveYearAvgDividendYield", "N/A"),
            },
            "profitability": {
                "NetIncome": net_income.iloc[0],
                "TotalRevenue": total_revenue.iloc[0],
                "CostOfRevenue": cost_of_revenue.iloc[0],
                "GrossProfit": gross_profit.iloc[0],
                "OperatingExpenses": operating_expenses.iloc[0],
                "CostOfRevenueCAGR": cagr_cost_of_revenue,
                "TotalRevenueCAGR": cagr_total_revenue,
                "OperatingExpensesCAGR": cagr_operating_expenses,
                # "CostOfRevenueYOY": yoy_cost_of_revenue,
                # "TotalRevenueYOY": yoy_total_revenue,
            },
            "liquidity": {
                # Actual
                "TotalAssets": total_assets.iloc[0],
                "TotalLiabilities": total_liabilities.iloc[0],
                "NetWorth": net_worth,
                "CashCashEquivalents": cash_cash_equivalents.iloc[0],
                # Short Term 1y
                "ShortTermDebtCoverage" : short_term_debt_coverage.iloc[0],
                "CurrentAssets": current_assets.iloc[0],
                "CurrentLiabilities": current_liabilities.iloc[0],
                # Long Term
                "LongTermDebtCoverage" : long_term_debt_coverage.iloc[0],
                "NonCurrentAssets": non_current_assets.iloc[0],
                "NonCurrentLiabilities": non_current_liabilities.iloc[0],
                # Growth
                "TotalAssetsCAGR": cagr_total_assets,
                "TotalLiabilitiesCAGR": cagr_total_liabilities,
                # Stockholders Equity
                "StockholdersEquityCAGR": cagr_stockholder_equity,
                "StockholdersEquity": stockholders_equity.iloc[0],
            },
            "cashflow": {
                "FreeCashflow": free_cashflow.iloc[0],
                "OperatingCashflow": operating_cashflow.iloc[0],
                "CapitalExpenditure": capital_expenditure.iloc[0],
                "MarketCap": market_cap,
                "FreeCashflowYield": free_cashflow_yield,
            },
            "ratios": {
                # Health & Debt
                "CurrentRatio": current_ratio.iloc[0],
                "CurrentRatioCAGR": cagr_current_ratio,
                "CashRatio": cash_ratio.iloc[0],
                "CashRatioCAGR": cagr_cash_ratio,
                # Margins
                "GrossMargin": gross_margin.iloc[0],
                # "GrossMarginCAGR": cagr_gross_margin,
                "OperatingMargin": operating_margin.iloc[0],
                "OperatingMarginCAGR": cagr_operating_margin,
                "ProfitMargin": profit_margin.iloc[0],
                "ProfitMarginCAGR": cagr_profit_margin,
                "ReturnOnEquity": return_on_equity.iloc[0],
                "ReturnOnEquityCAGR": cagr_return_on_equity,
            },
            "market_risk_and_sentiment": {
                "beta": yahoo_symbol_info.get("beta", "N/A"),
                "auditRisk": yahoo_symbol_info.get("auditRisk", "N/A"),
                "boardRisk": yahoo_symbol_info.get("boardRisk", "N/A"),
                "sharesPercentSharesOut": yahoo_symbol_info.get("sharesPercentSharesOut", "N/A"),
                "recommendationMean": yahoo_symbol_info.get("recommendationMean", "N/A"),
                "targetMeanPrice": yahoo_symbol_info.get("targetMeanPrice", "N/A")
            }
        }
        return yahoo_symbol_fundamental_info


In [58]:
get_symbol_fundamental_info("TNXP")

{'valuation': {'trailingPE': 'N/A',
  'sectorTrailingPE': 29.6347,
  'forwardPE': -0.9770301,
  'PEGRatio': None},
 'dividends': {'divCoverageRate': 'N/A',
  'dividendYield': 'N/A',
  'fiveYearAvgDividendYield': 'N/A'},
 'profitability': {'NetIncome': -130036000.0,
  'TotalRevenue': 10094000.0,
  'CostOfRevenue': 7765000.0,
  'GrossProfit': 2329000.0,
  'OperatingExpenses': 77479000.0,
  'CostOfRevenueCAGR': 'N/A',
  'TotalRevenueCAGR': 'N/A',
  'OperatingExpensesCAGR': -5.671712766156556},
 'liquidity': {'TotalAssets': 162890000.0,
  'TotalLiabilities': 23332000.0,
  'NetWorth': 139558000.0,
  'CashCashEquivalents': 98776000.0,
  'ShortTermDebtCoverage': 100695000.0,
  'CurrentAssets': 119002000.0,
  'CurrentLiabilities': 18307000.0,
  'LongTermDebtCoverage': 38863000.0,
  'NonCurrentAssets': 43888000.0,
  'NonCurrentLiabilities': 5025000.0,
  'TotalAssetsCAGR': -12.228697925291732,
  'TotalLiabilitiesCAGR': 1.6975669558642714,
  'StockholdersEquityCAGR': -13.909070088046304,
  'Stock

In [11]:
# https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


# ### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
# import requests

# url = "https://api.binance.com/api/v3/ticker/price"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)

# if response.status_code == 200:
#     symbol = response.json()["symbol"]
#     price = response.json()["price"]
# else:
#     error = response.status_code

In [ ]:
# from decimal import Decimal
# from datetime import datetime


# def get_crypto_symbol_24h(symbol : str):

#     url = "https://api.binance.com/api/v3/ticker/24hr"
#     params = {"symbol": symbol}

#     response = requests.get(url, params=params)

#     if response.status_code == 200:
#         priceChangePercent = Decimal(response.json()["priceChangePercent"])
#         weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
#         prevClosePrice = Decimal(response.json()["prevClosePrice"])
#         priceChange = Decimal(response.json()["priceChange"])
#         lastPrice = Decimal(response.json()["lastPrice"])
#         lastQty = Decimal(response.json()["lastQty"])
#         bidPrice = Decimal(response.json()["bidPrice"])
#         bidQty = Decimal(response.json()["bidQty"])
#         askPrice = Decimal(response.json()["askPrice"])
#         askQty = Decimal(response.json()["askQty"])
#         openPrice = Decimal(response.json()["openPrice"])
#         highPrice = Decimal(response.json()["highPrice"])
#         lowPrice = Decimal(response.json()["lowPrice"])
#         volume = Decimal(response.json()["volume"])
#         quoteVolume = Decimal(response.json()["quoteVolume"])
#         openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         firstId = response.json()["firstId"]
#         lastId = response.json()["lastId"]
#         count = response.json()["count"]
#     else:
#         print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
# url = "https://api.binance.com/api/v3/openOrders"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)
# response.json()
